# imports and defs

In [13]:
import numpy as np
import bcsd_module as bb
import matplotlib.pyplot as plt
import utility_module_2_v_3 as uu
from datetime import datetime
import pandas as pd
from tqdm import tqdm

In [14]:
def check_for_nans_infs_zeros(data_sts):
    print('Nans:',np.sum(np.isnan(data_sts)))
    print('Infs:' ,np.sum(np.isinf(data_sts)))
    print('Zeros:',np.sum((data_sts==0)))
    

In [8]:
def printr(text):
    print("\x1b[31m" + text+ "\x1b[0m")

In [9]:
## check dimensions and plot ts
def data_check(data_sts,lat,lon,metadata,Np= 12):
    check_for_nans_infs_zeros(data_sts)
    map_bounds=metadata['data_bounds'] #[50 , 50, 110, 0] #ltrb
    mp_spacing=10

    plt.figure(figsize = (10,2))

    plt.subplot(121)
    uu.map_plot_cyl(np.nanmean(data_sts[:,:,:],axis=0),lat,lon,map_bounds,mp_spacing=10)
    plt.title('Mean')

    plt.subplot(122)
    plt.plot(data_sts[:Np*5,int(lat.shape[0]/2),int(lon.shape[0]/2)])
    plt.plot(data_sts[-Np*5:,int(lat.shape[0]/2),int(lon.shape[0]/2)])


In [10]:
print(data_check)

<function data_check at 0x7fc5f46ba820>


# Exp details

In [11]:
exp_names =['India_P_BCSDd',
            # 'India_P_BCSDd_eqm',
            # 'India_P_BCSDd_CH',
            # 'India_P_BCSDd_CH_eqm',
            'India_T_BCSDd',
            # 'India_T_BCSDd_eqm',
            # 'India_P_BCSDd_NY',
            # 'India_P_BCSDd_NY_eqm',

           ]

SD_types = ['PREC',
            # 'PREC','PREC','PREC',
            'TEMP',
            # 'TEMP','PREC','PREC',
           ]
bc_methods = [bb.gamma_quantile_mapping,
              # bb.empirical_quantile_mapping,
              # bb.gamma_quantile_mapping,bb.empirical_quantile_mapping,
              bb.normal_quantile_mapping,
              # bb.empirical_quantile_mapping,
              # bb.gamma_quantile_mapping,bb.empirical_quantile_mapping,
             ]
var_types = ['pre',
             # 'pre','pre','pre',
             'tas',
             # 'tas','pre','pre'
            ]

df = pd.DataFrame(list(zip(exp_names, SD_types,bc_methods,var_types)),
               columns =['exp_names', 'SD_types','bc_methods', 'var_types'])
df

,exp_names,SD_types,bc_methods,var_types
0,India_P_BCSDd,PREC,<function gamma_quantile_mapping at 0x7fc5f46b...,pre
1,India_T_BCSDd,TEMP,<function normal_quantile_mapping at 0x7fc63f8...,tas


# Runs

In [16]:

gcm_name ='CanESM2'
for itr in range(len(df)):
    exp_name = df['exp_names'][itr]
    SD_type = df['SD_types'][itr]
    bc_method = df['bc_methods'][itr]
    var_type = df['var_types'][itr]
    
    path_out = '/home/vikram/BCSD_data/' + exp_name  + '/'
    metadata = np.load(path_out+'bcsd_imd_metadata_'+gcm_name + '.npy',allow_pickle=True).item()
    pre_obs_train = np.load(path_out+'data_obs_imd_train'+ '.npy')
    
    pre_gcm_train = np.load(path_out+'data_gcm_'+gcm_name+'_imd_train.npy')
    pre_gcm_test= np.load(path_out+'data_gcm_'+gcm_name+'_imd_pred.npy') 
    



    print("Experiment Name:",exp_name)

    if np.sum(np.isnan(pre_obs_train)) > 0:
        printr(str(np.sum(np.isnan(pre_obs_train))) + " nans fixed with average")
        pre_obs_train[np.isnan(pre_obs_train)] =np.nanmean(pre_obs_train)

    path_out = metadata['path_out']


    print("Started at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
    print('GCM name : ', gcm_name)
    print('Output path : ',path_out)
    
    for rnd_iter  in tqdm(range(100)):
        rdn_train=  np.random.randn(pre_gcm_train.shape[0],pre_gcm_train.shape[1],pre_gcm_train.shape[2])
        rdn_test=  np.random.randn(pre_gcm_test.shape[0],pre_gcm_test.shape[1],pre_gcm_test.shape[2])        
        if SD_type == 'PREC':
            rdn_train =np.exp(rdn_train) 
            rdn_test =np.exp(rdn_test) 
            print(SD_type)
        else:
            pass
        
        data_bcsd_rnd = bb.bcsd(var_type + '_rnd_new_'+ str(rnd_iter),pre_obs_train,rdn_train,
                                rdn_test,metadata,bc_method,
                                temporal_res = "Daily",sd_type =SD_type,mode = 1)
    
    print("Finished at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))    
    
    


Experiment Name: India_P_BCSDd
Started at: 12/09/2023 19:45:25
GCM name :  CanESM2
Output path :  /home/vikram/BCSD_data/India_P_BCSDd/


  0%|                                             | 0/100 [00:00<?, ?it/s]

PREC
Bias Correction Spatially - daily:
Training years : 50.0 == 50.0
Testing years : 105.0
Bias correction fucntion : <function gamma_quantile_mapping at 0x7fc5f46ba040>
Input shape (18250, 14, 14) (18250, 14, 14) (38325, 14, 14)
14 14



100%|█████████████████████████████████████| 14/14 [04:23<00:00, 18.82s/it]


Output shape (38325, 14, 14)
Spatial Disaggregation - Daily (NCLIM ==  365 )
SD Type - PREC
No of timesteps  38325
SD Input shape (18250, 33, 35) (38325, 14, 14)


...

SD Output shape (20075, 31, 31)
Bias Correction Spatially - daily:
Training years : 25.0 == 25.0
Testing years : 55.0
Bias correction fucntion : <function normal_quantile_mapping at 0x7fc63f87f310>
Input shape (9125, 14, 14) (9125, 14, 14) (20075, 14, 14)
14 14



100%|█████████████████████████████████████| 14/14 [00:14<00:00,  1.01s/it]


Output shape (20075, 14, 14)
Spatial Disaggregation - Daily (NCLIM ==  365 )
SD Type - PREC
No of timesteps  20075
SD Input shape (9125, 31, 31) (20075, 14, 14)



100%|███████████████████████████████████| 100/100 [37:27<00:00, 22.48s/it]

SD Output shape (20075, 31, 31)
Finished at: 13/09/2023 04:01:08


## Indian Precipitation

In [ ]:
['India_P_BCSDd',
'India_P_BCSDd_CH',
'India_P_BCSDd_CH_eqm',
'India_P_BCSDd_eqm',
'India_T_BCSDd',
'India_T_BCSDd_eqm']



## Load data

In [ ]:
path_out = '/home/vikram/BCSD_data/India_P_BCSDd_CH_eqm/'
gcm_name ='CanESM2'
metadata = np.load(path_out+'bcsd_imd_metadata_'+gcm_name + '.npy',allow_pickle=True).item()
pre_obs_train = np.load(path_out+'data_obs_imd_train'+ '.npy')
pre_gcm_train = np.load(path_out+'data_gcm_'+gcm_name+'_imd_train.npy')
pre_gcm_test= np.load(path_out+'data_gcm_'+gcm_name+'_imd_pred.npy') 
path_out = metadata['path_out']

rdn_train=  np.random.randn(pre_gcm_train.shape[0],pre_gcm_train.shape[1],pre_gcm_train.shape[2])*np.nanmean(pre_gcm_train)
rdn_test=  np.random.randn(pre_gcm_test.shape[0],pre_gcm_test.shape[1],pre_gcm_test.shape[2])*np.nanmean(pre_gcm_test)
rdn_train[rdn_train<0]=0
rdn_test[rdn_test<0]=0


data_check(pre_obs_train, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(pre_gcm_train, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(rdn_test, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)


## Run BCSD


In [ ]:
print("Started at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('GCM name : ', gcm_name)
print('Output path : ',path_out)

data_bcsd_pred = bb.bcsd('pre_'+gcm_name,pre_obs_train,pre_gcm_train,pre_gcm_test,metadata,
                         bb.empirical_quantile_mapping,temporal_res = "Daily",sd_type ="PREC",mode = 1)

data_bcsd_rnd = bb.bcsd('pre_rnd_0',pre_obs_train,rdn_train,
                        rdn_test,metadata,bb.empirical_quantile_mapping,
                        temporal_res = "Daily",sd_type ="PREC",mode = 1)
print("Finished at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


## Check

In [ ]:
data_check(pre_obs_train, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(pre_gcm_train, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(rdn_test, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(data_bcsd_pred, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(data_bcsd_rnd, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)


## Indian Precipitation CH

## Load data

In [ ]:
path_out = '/home/vikram/BCSD_data/India_P_BCSDd_salvi_new_emp/'
gcm_name ='CanESM2'
metadata = np.load(path_out+'bcsd_imd_metadata_'+gcm_name + '.npy',allow_pickle=True).item()
pre_obs_train = np.load(path_out+'data_obs_imd_train'+ '.npy')
pre_gcm_train = np.load(path_out+'data_gcm_'+gcm_name+'_imd_train.npy')
pre_gcm_test= np.load(path_out+'data_gcm_'+gcm_name+'_imd_pred.npy')
                        
path_out = metadata['path_out']

## Run BCSD


In [ ]:
print("Started at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('GCM name : ', gcm_name)
print('Output path : ',path_out)

data_bcsd_pred = bb.bcsd('pre_'+gcm_name,pre_obs_train,pre_gcm_train,pre_gcm_test,metadata,
                         bb.empirical_quantile_mapping,temporal_res = "Daily",sd_type ="PREC",mode = 1)

rdn_train=  np.random.randn(pre_gcm_train.shape[0],pre_gcm_train.shape[1],pre_gcm_train.shape[2])*np.nanmean(pre_gcm_train)
rdn_test=  np.random.randn(pre_gcm_test.shape[0],pre_gcm_test.shape[1],pre_gcm_test.shape[2])*np.nanmean(pre_gcm_test)
rdn_train[rdn_train<0]=0
rdn_test[rdn_test<0]=0
data_bcsd_rnd = bb.bcsd('pre_rnd_0',pre_obs_train,
                        rdn_train,rdn_test,metadata,bb.empirical_quantile_mapping,
                        temporal_res = "Daily",sd_type ="PREC",mode = 1)

print("Finished at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


## Check

In [ ]:
data_check(pre_obs_train, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(pre_gcm_train, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(data_bcsd_pred, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(rdn_test, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(data_bcsd_rnd, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)


## Indian Temperature

## Load data

In [ ]:
path_out = '/home/vikram/BCSD_data/India_T_BCSDd_new/'
gcm_name ='CanESM2'
metadata = np.load(path_out+'bcsd_imd_metadata_'+gcm_name + '.npy',allow_pickle=True).item()
tas_obs_train = np.load(path_out+'data_obs_imd_train'+ '.npy')
tas_gcm_train = np.load(path_out+'data_gcm_'+gcm_name+'_imd_train.npy')
tas_gcm_test= np.load(path_out+'data_gcm_'+gcm_name+'_imd_pred.npy')
                         
      
        
path_out = metadata['path_out']

In [ ]:
metadata['data_bounds']=[65, 40, 100, 5]


## Run BCSD


In [ ]:
print("Started at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
print('GCM name : ', gcm_name)
print('Output path : ',path_out)

data_bcsd_pred = bb.bcsd('tas_'+gcm_name+'_cru',tas_obs_train,tas_gcm_train,tas_gcm_test,metadata,
                         bb.empirical_quantile_mapping,temporal_res = "Daily",sd_type ="TEMP",mode = 1)

rdn_train=  np.random.randn(tas_gcm_train.shape[0],tas_gcm_train.shape[1],tas_gcm_train.shape[2])*np.nanmean(tas_gcm_train)
rdn_test=  np.random.randn(tas_gcm_test.shape[0],tas_gcm_test.shape[1],tas_gcm_test.shape[2])*np.nanmean(tas_gcm_test)
rdn_train[rdn_train<0]=0
rdn_test[rdn_test<0]=0
data_bcsd_rnd = bb.bcsd('tas_rnd_0_cru',tas_obs_train,rdn_train,rdn_test,metadata,bb.empirical_quantile_mapping,
                        temporal_res = "Daily",sd_type ="TEMP",mode = 1)

print("Finished at:", datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


In [ ]:
tas_obs_train.shape

## Check

In [ ]:
data_check(tas_obs_train, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(tas_gcm_train, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(data_bcsd_pred, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
data_check(rdn_test, metadata['lat_gcm'], metadata['lon_gcm'],metadata,Np=365)
data_check(data_bcsd_rnd, metadata['lat_obs'], metadata['lon_obs'],metadata,Np=365)
